In [141]:

%matplotlib inline

import sklearn
from sklearn.feature_selection import SelectKBest




import gc
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import defaultdict

plt.style.use('default') # Make the graphs a bit prettier
plt.rcParams['figure.figsize'] = (15, 5)

In [142]:
station = pd.read_csv('station.csv', low_memory=False)
trip_test = pd.read_csv('trip_test.csv', low_memory=False)
trip_train = pd.read_csv('trip_train.csv', low_memory=False)
weather = pd.read_csv('weather.csv', low_memory=False)


In [143]:
del weather['zip_code']


In [144]:
weather.columns


Index(['date', 'max_temperature_f', 'mean_temperature_f', 'min_temperature_f',
       'max_dew_point_f', 'mean_dew_point_f', 'min_dew_point_f',
       'max_humidity', 'mean_humidity', 'min_humidity',
       'max_sea_level_pressure_inches', 'mean_sea_level_pressure_inches',
       'min_sea_level_pressure_inches', 'max_visibility_miles',
       'mean_visibility_miles', 'min_visibility_miles', 'max_wind_Speed_mph',
       'mean_wind_speed_mph', 'max_gust_speed_mph', 'precipitation_inches',
       'cloud_cover', 'events', 'wind_dir_degrees'],
      dtype='object')

In [145]:
trip_train = pd.merge(trip_train, station[['id', 'lat', 'long']], left_on='start_station_id', right_on= 'id' ,how='left')


In [146]:
trip_train.drop('id_y', axis=1, inplace=True)
trip_train.rename(columns={'id_x' : 'id', 'lat': 'start_station_lat','long': 'start_station_long'}, inplace=True)

In [147]:
trip_train = pd.merge(trip_train, station[['id', 'lat', 'long']], left_on='end_station_id', right_on= 'id' ,how='left')
trip_train

,id_x,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,start_station_lat,start_station_long,id_y,lat,long
0,907649,396,8/27/2015 8:36,Harry Bridges Plaza (Ferry Building),50,8/27/2015 8:43,Embarcadero at Sansome,60,187,Subscriber,94602,37.795392,-122.394203,60,37.804770,-122.403234
1,384043,636,7/28/2014 22:06,Market at 10th,67,7/28/2014 22:17,Washington at Kearny,46,417,Subscriber,94133,37.776619,-122.417385,46,37.795425,-122.404767
2,316176,334,6/9/2014 8:42,Market at Sansome,77,6/9/2014 8:47,2nd at Folsom,62,281,Subscriber,94107,37.789625,-122.400811,62,37.785299,-122.396236
3,618874,666,1/26/2015 16:55,San Francisco Caltrain 2 (330 Townsend),69,1/26/2015 17:07,Temporary Transbay Terminal (Howard at Beale),55,634,Subscriber,94602,37.776600,-122.395470,55,37.789756,-122.394643
4,910977,318,8/29/2015 15:09,Market at 10th,67,8/29/2015 15:14,Powell Street BART,39,607,Subscriber,94709,37.776619,-122.417385,39,37.783871,-122.408433
5,522083,337,10/30/2014 7:00,Townsend at 7th,65,10/30/2014 7:06,San Francisco Caltrain (Townsend at 4th),70,370,Subscriber,94107,37.771058,-122.402717,70,37.776617,-122.395260
6,880809,394,8/7/2015 17:11,2nd at South Park,64,8/7/2015 17:17,2nd at Townsend,61,443,Subscriber,94107,37.782259,-122.392738,61,37.780526,-122.390288
7,488938,766,10/8/2014 14:10,Powell at Post (Union Square),71,10/8/2014 14:23,San Francisco Caltrain (Townsend at 4th),70,485,Subscriber,94107,37.788446,-122.408499,70,37.776617,-122.395260
8,899522,531,8/21/2015 7:53,2nd at Folsom,62,8/21/2015 8:02,San Francisco Caltrain (Townsend at 4th),70,603,Subscriber,94549,37.785299,-122.396236,70,37.776617,-122.395260
9,737380,267,4/23/2015 6:36,Market at 4th,76,4/23/2015 6:40,Mechanics Plaza (Market at Battery),75,86,Customer,21202,37.786305,-122.404966,75,37.791300,-122.399051


In [148]:
trip_train.drop('id_y', axis=1, inplace=True)
trip_train.rename(columns={'id_x' : 'id', 'lat': 'end_station_lat','long': 'end_station_long'}, inplace=True)
trip_train

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,start_station_lat,start_station_long,end_station_lat,end_station_long
0,907649,396,8/27/2015 8:36,Harry Bridges Plaza (Ferry Building),50,8/27/2015 8:43,Embarcadero at Sansome,60,187,Subscriber,94602,37.795392,-122.394203,37.804770,-122.403234
1,384043,636,7/28/2014 22:06,Market at 10th,67,7/28/2014 22:17,Washington at Kearny,46,417,Subscriber,94133,37.776619,-122.417385,37.795425,-122.404767
2,316176,334,6/9/2014 8:42,Market at Sansome,77,6/9/2014 8:47,2nd at Folsom,62,281,Subscriber,94107,37.789625,-122.400811,37.785299,-122.396236
3,618874,666,1/26/2015 16:55,San Francisco Caltrain 2 (330 Townsend),69,1/26/2015 17:07,Temporary Transbay Terminal (Howard at Beale),55,634,Subscriber,94602,37.776600,-122.395470,37.789756,-122.394643
4,910977,318,8/29/2015 15:09,Market at 10th,67,8/29/2015 15:14,Powell Street BART,39,607,Subscriber,94709,37.776619,-122.417385,37.783871,-122.408433
5,522083,337,10/30/2014 7:00,Townsend at 7th,65,10/30/2014 7:06,San Francisco Caltrain (Townsend at 4th),70,370,Subscriber,94107,37.771058,-122.402717,37.776617,-122.395260
6,880809,394,8/7/2015 17:11,2nd at South Park,64,8/7/2015 17:17,2nd at Townsend,61,443,Subscriber,94107,37.782259,-122.392738,37.780526,-122.390288
7,488938,766,10/8/2014 14:10,Powell at Post (Union Square),71,10/8/2014 14:23,San Francisco Caltrain (Townsend at 4th),70,485,Subscriber,94107,37.788446,-122.408499,37.776617,-122.395260
8,899522,531,8/21/2015 7:53,2nd at Folsom,62,8/21/2015 8:02,San Francisco Caltrain (Townsend at 4th),70,603,Subscriber,94549,37.785299,-122.396236,37.776617,-122.395260
9,737380,267,4/23/2015 6:36,Market at 4th,76,4/23/2015 6:40,Mechanics Plaza (Market at Battery),75,86,Customer,21202,37.786305,-122.404966,37.791300,-122.399051


In [149]:
trip_train['start_station_lat'] = trip_train[pd.to_numeric(trip_train.start_station_lat, errors='coerce').notnull()]['start_station_lat']
trip_train['start_station_long'] = trip_train[pd.to_numeric(trip_train.start_station_long, errors='coerce').notnull()]['start_station_long']
trip_train['end_station_lat'] = trip_train[pd.to_numeric(trip_train.end_station_lat, errors='coerce').notnull()]['end_station_lat']
trip_train['end_station_long'] = trip_train[pd.to_numeric(trip_train.end_station_long, errors='coerce').notnull()]['end_station_long']


In [150]:
from math import radians, cos, sin, asin, sqrt

#distancia entre dos estaciones 
def distanciaEntreEstaciones(long1, lat1, long2, lat2):
    long1, lat1, long2, lat2 = map(radians, [long1, lat1, long2, lat2])
    dLong = long2 - long1 
    dLat = lat2 - lat1 
    a = sin(dLat/2)**2 + cos(lat1) * cos(lat2) * sin(dLong/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    return km

distancia = trip_train.apply(lambda x: distanciaEntreEstaciones(x[11], x[12], x[13], x[14]), axis = 1)
trip_train['distancia_entre_estaciones'] = distancia


In [151]:
trip_train

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,start_station_lat,start_station_long,end_station_lat,end_station_long,distancia_entre_estaciones
0,907649,396,8/27/2015 8:36,Harry Bridges Plaza (Ferry Building),50,8/27/2015 8:43,Embarcadero at Sansome,60,187,Subscriber,94602,37.795392,-122.394203,37.804770,-122.403234,1.148453
1,384043,636,7/28/2014 22:06,Market at 10th,67,7/28/2014 22:17,Washington at Kearny,46,417,Subscriber,94133,37.776619,-122.417385,37.795425,-122.404767,1.794651
2,316176,334,6/9/2014 8:42,Market at Sansome,77,6/9/2014 8:47,2nd at Folsom,62,281,Subscriber,94107,37.789625,-122.400811,37.785299,-122.396236,0.569924
3,618874,666,1/26/2015 16:55,San Francisco Caltrain 2 (330 Townsend),69,1/26/2015 17:07,Temporary Transbay Terminal (Howard at Beale),55,634,Subscriber,94602,37.776600,-122.395470,37.789756,-122.394643,0.788625
4,910977,318,8/29/2015 15:09,Market at 10th,67,8/29/2015 15:14,Powell Street BART,39,607,Subscriber,94709,37.776619,-122.417385,37.783871,-122.408433,1.084530
5,522083,337,10/30/2014 7:00,Townsend at 7th,65,10/30/2014 7:06,San Francisco Caltrain (Townsend at 4th),70,370,Subscriber,94107,37.771058,-122.402717,37.776617,-122.395260,0.892320
6,880809,394,8/7/2015 17:11,2nd at South Park,64,8/7/2015 17:17,2nd at Townsend,61,443,Subscriber,94107,37.782259,-122.392738,37.780526,-122.390288,0.291147
7,488938,766,10/8/2014 14:10,Powell at Post (Union Square),71,10/8/2014 14:23,San Francisco Caltrain (Townsend at 4th),70,485,Subscriber,94107,37.788446,-122.408499,37.776617,-122.395260,1.631116
8,899522,531,8/21/2015 7:53,2nd at Folsom,62,8/21/2015 8:02,San Francisco Caltrain (Townsend at 4th),70,603,Subscriber,94549,37.785299,-122.396236,37.776617,-122.395260,0.528155
9,737380,267,4/23/2015 6:36,Market at 4th,76,4/23/2015 6:40,Mechanics Plaza (Market at Battery),75,86,Customer,21202,37.786305,-122.404966,37.791300,-122.399051,0.721470


In [152]:
del trip_train['start_station_lat']
del trip_train['start_station_long']
del trip_train['end_station_lat']
del trip_train['end_station_long']

In [153]:
fechasStart = trip_train['start_date'].map(lambda x: (x.split(' ')[0]))
trip_train['startDate'] =  fechasStart
fechasEnd = trip_train['end_date'].map(lambda x: (x.split(' ')[0]))
trip_train['endDate'] =  fechasEnd

In [154]:
#separamos la fecha por / y eliminamos todo lo q hay mas a la derecha del espacio despues del año
fechas = trip_train['start_date'].map(lambda x: (x.split('/')[0], x.split('/')[1], x.split('/')[2].split(' ')[0]))

#convertimos cada tupla de año mes dia a dia de semana ylo guardamos en un nuevo campo 
trip_train['startDate_weekday'] = fechas.map(lambda x: dt.date(int(x[2]),int(x[0]),int(x[1])).strftime("%A"))

#separamos la fecha por / y eliminamos todo lo q hay mas a la derecha del espacio despues del año
fechas = trip_train['end_date'].map(lambda x: (x.split('/')[0], x.split('/')[1], x.split('/')[2].split(' ')[0]))

#convertimos cada tupla de año mes dia a dia de semana ylo guardamos en un nuevo campo 
trip_train['endDate_weekday'] = fechas.map(lambda x: dt.date(int(x[2]),int(x[0]),int(x[1])).strftime("%A"))

In [155]:
trip_train[trip_train['startDate'] != trip_train['endDate']]

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,distancia_entre_estaciones,startDate,endDate,startDate_weekday,endDate_weekday
62,246208,64803,4/12/2014 17:23,Grant Avenue at Columbus Avenue,73,4/13/2014 11:23,2nd at Townsend,61,364,Customer,7302,2.167697,4/12/2014,4/13/2014,Saturday,Sunday
410,402371,59644,8/10/2014 23:10,Yerba Buena Center of the Arts (3rd @ Howard),68,8/11/2014 15:44,Golden Gate at Polk,59,222,Customer,94102,1.965957,8/10/2014,8/11/2014,Sunday,Monday
984,566595,15091,12/4/2014 22:46,San Francisco City Hall,58,12/5/2014 2:57,Market at 10th,67,387,Customer,92705,0.153497,12/4/2014,12/5/2014,Thursday,Friday
1245,187216,30023,2/17/2014 23:27,Harry Bridges Plaza (Ferry Building),50,2/18/2014 7:47,Embarcadero at Bryant,54,584,Subscriber,94107,0.844848,2/17/2014,2/18/2014,Monday,Tuesday
1331,658718,241766,2/26/2015 2:42,Santa Clara at Almaden,4,2/28/2015 21:52,San Jose Diridon Caltrain Station,2,46,Customer,95126,0.804350,2/26/2015,2/28/2015,Thursday,Saturday
1397,32352,534,9/23/2013 23:57,Embarcadero at Sansome,60,9/24/2013 0:06,Mechanics Plaza (Market at Battery),75,290,Subscriber,94102,0.927041,9/23/2013,9/24/2013,Monday,Tuesday
1654,760403,68367,5/9/2015 23:08,Davis at Jackson,42,5/10/2015 18:08,Mechanics Plaza (Market at Battery),75,523,Customer,94015,0.362559,5/9/2015,5/10/2015,Saturday,Sunday
1664,825527,46016,6/28/2015 13:30,South Van Ness at Market,66,6/29/2015 2:17,Townsend at 7th,65,502,Customer,81475,1.818154,6/28/2015,6/29/2015,Sunday,Monday
1675,635323,71356,2/8/2015 13:47,Embarcadero at Vallejo,48,2/9/2015 9:36,2nd at Townsend,61,442,Customer,979,1.474964,2/8/2015,2/9/2015,Sunday,Monday
1935,535575,458,11/7/2014 23:53,Mountain View Caltrain Station,28,11/8/2014 0:01,Castro Street and El Camino Real,32,102,Subscriber,94040,0.919212,11/7/2014,11/8/2014,Friday,Saturday


In [156]:
#separamos en dos campos nuevos. fecha en formato yyyymmdd y hora del dia en minutos
fechasStart = trip_train['start_date'].map(lambda x: (int(dt.datetime.strptime(x.split(' ')[0], "%m/%d/%Y").strftime("%Y%m%d"))))
horasStart = trip_train['start_date'].map(lambda x: (x.split(' ')[1]))
horasStart = horasStart.map(lambda x: (int(x.split(':')[0]) * 60 *60+ int(x.split(':')[1])*60))

fechasEnd = trip_train['end_date'].map(lambda x: (int(dt.datetime.strptime(x.split(' ')[0], "%m/%d/%Y").strftime("%Y%m%d"))))
horasEnd = trip_train['end_date'].map(lambda x: (x.split(' ')[1]))
horasEnd = horasEnd.map(lambda x: (int(x.split(':')[0]) * 60 *60+ int(x.split(':')[1])*60))

In [157]:
#ver dia y noche como afecta

#separamos la fecha por / y eliminamos todo lo q hay mas a la derecha del espacio despues del año
fechas = trip_train['start_date'].map(lambda x: (x.split('/')[0], x.split('/')[1], x.split('/')[2].split(' ')[0]))

horas = trip_train['start_date'].map(lambda x:x.split('/')[2].split(' ')[1].split(':')[0])
#convertimos cada tupla de año mes dia a dia de semana ylo guardamos en un nuevo campo 

fechas = fechas.map(lambda x: dt.date(int(x[2]),int(x[0]),int(x[1])).strftime("%m"))

i = 0
deNoche = [0] * fechas.size
for line in fechas:
    if(((int(line) < 4) & (int(line) > 0)) | ((int(line) > 9)&(int(line) < 13))):
        if((int(horas[i]) <18) & (int(horas[i]) > 7)):
            deNoche[i] = 0
        else:
            deNoche[i] = 1
    else:
        if((int(horas[i]) <20) & (int(horas[i]) > 6)):
            deNoche[i] = 0
        else:
            deNoche[i] = 1
    i = i + 1 
        
trip_train['startDate_deNoche'] = deNoche

#ver dia y noche como afecta

#separamos la fecha por / y eliminamos todo lo q hay mas a la derecha del espacio despues del año
fechas = trip_train['end_date'].map(lambda x: (x.split('/')[0], x.split('/')[1], x.split('/')[2].split(' ')[0]))

horas = trip_train['end_date'].map(lambda x:x.split('/')[2].split(' ')[1].split(':')[0])
#convertimos cada tupla de año mes dia a dia de semana ylo guardamos en un nuevo campo 

fechas = fechas.map(lambda x: dt.date(int(x[2]),int(x[0]),int(x[1])).strftime("%m"))

i = 0
deNoche = [0] * fechas.size
for line in fechas:
    if(((int(line) < 4) & (int(line) > 0)) | ((int(line) > 9)&(int(line) < 13))):
        if((int(horas[i]) <18) & (int(horas[i]) > 7)):
            deNoche[i] = 0
        else:
            deNoche[i] = 1
    else:
        if((int(horas[i]) <20) & (int(horas[i]) > 6)):
            deNoche[i] = 0
        else:
            deNoche[i] = 1
    i = i + 1 
        
trip_train['endDate_deNoche'] = deNoche

In [158]:
trip_train = pd.merge(trip_train, weather, left_on='startDate', right_on= 'date' ,how='left')
trip_train

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,...,max_visibility_miles,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees
0,907649,396,8/27/2015 8:36,Harry Bridges Plaza (Ferry Building),50,8/27/2015 8:43,Embarcadero at Sansome,60,187,Subscriber,...,10.0,10.0,10.0,17.0,6.0,29.0,0,2.0,NaN,272.0
1,907649,396,8/27/2015 8:36,Harry Bridges Plaza (Ferry Building),50,8/27/2015 8:43,Embarcadero at Sansome,60,187,Subscriber,...,10.0,10.0,10.0,14.0,3.0,20.0,0,1.0,NaN,309.0
2,907649,396,8/27/2015 8:36,Harry Bridges Plaza (Ferry Building),50,8/27/2015 8:43,Embarcadero at Sansome,60,187,Subscriber,...,20.0,13.0,10.0,17.0,6.0,NaN,0,0.0,NaN,349.0
3,907649,396,8/27/2015 8:36,Harry Bridges Plaza (Ferry Building),50,8/27/2015 8:43,Embarcadero at Sansome,60,187,Subscriber,...,10.0,10.0,10.0,22.0,5.0,25.0,0,0.0,NaN,356.0
4,907649,396,8/27/2015 8:36,Harry Bridges Plaza (Ferry Building),50,8/27/2015 8:43,Embarcadero at Sansome,60,187,Subscriber,...,10.0,10.0,10.0,23.0,6.0,29.0,0,3.0,NaN,313.0
5,384043,636,7/28/2014 22:06,Market at 10th,67,7/28/2014 22:17,Washington at Kearny,46,417,Subscriber,...,10.0,10.0,10.0,21.0,10.0,24.0,0,6.0,NaN,316.0
6,384043,636,7/28/2014 22:06,Market at 10th,67,7/28/2014 22:17,Washington at Kearny,46,417,Subscriber,...,10.0,10.0,10.0,14.0,5.0,NaN,0,4.0,NaN,345.0
7,384043,636,7/28/2014 22:06,Market at 10th,67,7/28/2014 22:17,Washington at Kearny,46,417,Subscriber,...,20.0,13.0,7.0,15.0,8.0,NaN,0,5.0,NaN,352.0
8,384043,636,7/28/2014 22:06,Market at 10th,67,7/28/2014 22:17,Washington at Kearny,46,417,Subscriber,...,10.0,10.0,10.0,16.0,5.0,21.0,0,1.0,NaN,339.0
9,384043,636,7/28/2014 22:06,Market at 10th,67,7/28/2014 22:17,Washington at Kearny,46,417,Subscriber,...,10.0,10.0,10.0,15.0,5.0,18.0,0,4.0,NaN,294.0


In [159]:
del trip_train['endDate']
del trip_train['startDate']
del trip_train['id']
del trip_train['start_station_name']
del trip_train['end_station_name']
del trip_train['start_date']
del trip_train['end_date']
del trip_train['date']
del trip_train['zip_code']


In [160]:
trip_train['start_date'] = fechasStart
trip_train['end_date'] = fechasEnd
trip_train['start_hour'] = horasStart
trip_train['end_hour'] = horasEnd

In [161]:
#convertimos subscription_type a 1 y 0 
trip_train['duration'] = trip_train[pd.to_numeric(trip_train.duration, errors='coerce').notnull()]['duration']
subscription_type = trip_train['subscription_type'].map(lambda x: 1 if x == 'Subscriber' else 0)
trip_train['subscription_type'] = subscription_type
trip_train.head()

,duration,start_station_id,end_station_id,bike_id,subscription_type,distancia_entre_estaciones,startDate_weekday,endDate_weekday,startDate_deNoche,endDate_deNoche,...,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,start_date,end_date,start_hour,end_hour
0,396,50,60,187,1,1.148453,Thursday,Thursday,0,0,...,6.0,29.0,0,2.0,NaN,272.0,20150827.0,20150827.0,30960.0,31380.0
1,396,50,60,187,1,1.148453,Thursday,Thursday,0,0,...,3.0,20.0,0,1.0,NaN,309.0,20140728.0,20140728.0,79560.0,80220.0
2,396,50,60,187,1,1.148453,Thursday,Thursday,0,0,...,6.0,NaN,0,0.0,NaN,349.0,20140609.0,20140609.0,31320.0,31620.0
3,396,50,60,187,1,1.148453,Thursday,Thursday,0,0,...,5.0,25.0,0,0.0,NaN,356.0,20150126.0,20150126.0,60900.0,61620.0
4,396,50,60,187,1,1.148453,Thursday,Thursday,0,0,...,6.0,29.0,0,3.0,NaN,313.0,20150829.0,20150829.0,54540.0,54840.0


In [162]:
trip_train['events'].value_counts()

Rain                 230599
Fog                   78439
Fog-Rain               9664
Rain-Thunderstorm      1451
rain                    916
Name: events, dtype: int64

In [163]:
trip_train['precipitation_inches'] = trip_train['precipitation_inches'].map(lambda x: 0.00001 if x == 'T' else x)

In [164]:
#convertimos las clases de la columna events
events = trip_train['events'].map(lambda x: 0 if ((x == 'rain') or (x == 'Rain')) else 1 if x == 'Fog' else 2 if x == 'Fog-Rain' else 3 if x == 'Rain-Thunderstorm' else 4)
trip_train['events'] = events
trip_train.head()

,duration,start_station_id,end_station_id,bike_id,subscription_type,distancia_entre_estaciones,startDate_weekday,endDate_weekday,startDate_deNoche,endDate_deNoche,...,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,start_date,end_date,start_hour,end_hour
0,396,50,60,187,1,1.148453,Thursday,Thursday,0,0,...,6.0,29.0,0,2.0,4,272.0,20150827.0,20150827.0,30960.0,31380.0
1,396,50,60,187,1,1.148453,Thursday,Thursday,0,0,...,3.0,20.0,0,1.0,4,309.0,20140728.0,20140728.0,79560.0,80220.0
2,396,50,60,187,1,1.148453,Thursday,Thursday,0,0,...,6.0,NaN,0,0.0,4,349.0,20140609.0,20140609.0,31320.0,31620.0
3,396,50,60,187,1,1.148453,Thursday,Thursday,0,0,...,5.0,25.0,0,0.0,4,356.0,20150126.0,20150126.0,60900.0,61620.0
4,396,50,60,187,1,1.148453,Thursday,Thursday,0,0,...,6.0,29.0,0,3.0,4,313.0,20150829.0,20150829.0,54540.0,54840.0


In [165]:
#convertimos las clases de los dias de semana
startDays = trip_train['startDate_weekday'].map(lambda x: 0 if x == 'Monday' else 1 if x == 'Thursday' 
                                             else 2 if x == 'Wednesday' else 3 if x == 'Tuesday' else 4 if x == 'Friday'
                                            else 5 if x == 'Saturday' else 6)
trip_train['startDate_weekday'] = startDays

endDays = trip_train['endDate_weekday'].map(lambda x: 0 if x == 'Monday' else 1 if x == 'Thursday' 
                                             else 2 if x == 'Wednesday' else 3 if x == 'Tuesday' else 4 if x == 'Friday'
                                            else 5 if x == 'Saturday' else 6)
trip_train['endDate_weekday'] = endDays
trip_train['startDate_weekday'].value_counts()

3    502050
2    492035
1    488755
0    475590
4    449575
5    183865
6    157935
Name: startDate_weekday, dtype: int64

In [166]:
trip_train['endDate_weekday'].value_counts()


3    502125
2    492120
1    488780
0    475830
4    448775
5    183705
6    158470
Name: endDate_weekday, dtype: int64

In [167]:
trip_train['events'].value_counts()

4    2428736
0     231515
1      78439
2       9664
3       1451
Name: events, dtype: int64

In [ ]:
trip_train.columns

In [21]:
del trip_train['bike_id']
del trip_train['mean_temperature_f']
del trip_train['min_temperature_f']
del trip_train['max_dew_point_f']
del trip_train['min_dew_point_f']
del trip_train['mean_dew_point_f']
del trip_train['mean_humidity']
del trip_train['min_humidity']
del trip_train['max_sea_level_pressure_inches']
del trip_train['min_sea_level_pressure_inches']
del trip_train['mean_sea_level_pressure_inches']
del trip_train['wind_dir_degrees']
del trip_train['mean_visibility_miles']
del trip_train['min_visibility_miles']
del trip_train['max_wind_Speed_mph']
del trip_train['max_humidity']

In [22]:
trip_train['is_true'] = np.random.uniform(0,1,len(trip_train)) <= 0.9
df = trip_train
train, test = df[df['is_true']==True], df[df['is_true']==False]


In [23]:
del train['is_true']
del test['is_true']
train


,duration,start_station_id,end_station_id,subscription_type,start_station_lat,start_station_long,end_station_lat,end_station_long,mean_temperature_f,mean_visibility_miles,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,start_date,end_date,start_hour,end_hour
0,396,50,60,1,37.795392,-122.394203,37.804770,-122.403234,72.0,10.0,6.0,29.0,0,2.0,4,20150827.0,20150827.0,30960.0,31380.0
1,396,50,60,1,37.795392,-122.394203,37.804770,-122.403234,76.0,10.0,3.0,20.0,0,1.0,4,20140728.0,20140728.0,79560.0,80220.0
3,396,50,60,1,37.795392,-122.394203,37.804770,-122.403234,76.0,10.0,5.0,25.0,0,0.0,4,20150126.0,20150126.0,60900.0,61620.0
4,396,50,60,1,37.795392,-122.394203,37.804770,-122.403234,78.0,10.0,6.0,29.0,0,3.0,4,20150829.0,20150829.0,54540.0,54840.0
5,636,67,46,1,37.776619,-122.417385,37.795425,-122.404767,67.0,10.0,10.0,24.0,0,6.0,4,20141030.0,20141030.0,25200.0,25560.0
6,636,67,46,1,37.776619,-122.417385,37.795425,-122.404767,70.0,10.0,5.0,NaN,0,4.0,4,20150807.0,20150807.0,61860.0,62220.0
7,636,67,46,1,37.776619,-122.417385,37.795425,-122.404767,71.0,13.0,8.0,NaN,0,5.0,4,20141008.0,20141008.0,51000.0,51780.0
8,636,67,46,1,37.776619,-122.417385,37.795425,-122.404767,70.0,10.0,5.0,21.0,0,1.0,4,20150821.0,20150821.0,28380.0,28920.0
9,636,67,46,1,37.776619,-122.417385,37.795425,-122.404767,71.0,10.0,5.0,18.0,0,4.0,4,20150423.0,20150423.0,23760.0,24000.0
10,334,77,62,1,37.789625,-122.400811,37.785299,-122.396236,68.0,9.0,10.0,25.0,0,1.0,4,20140520.0,20140520.0,32340.0,33120.0


In [169]:
q = trip_train
#w = test
q


,duration,start_station_id,end_station_id,bike_id,subscription_type,distancia_entre_estaciones,startDate_weekday,endDate_weekday,startDate_deNoche,endDate_deNoche,...,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,start_date,end_date,start_hour,end_hour
0,396,50,60,187,1,1.148453,1,1,0,0,...,6.0,29.0,0,2.0,4,272.0,20150827.0,20150827.0,30960.0,31380.0
1,396,50,60,187,1,1.148453,1,1,0,0,...,3.0,20.0,0,1.0,4,309.0,20140728.0,20140728.0,79560.0,80220.0
2,396,50,60,187,1,1.148453,1,1,0,0,...,6.0,NaN,0,0.0,4,349.0,20140609.0,20140609.0,31320.0,31620.0
3,396,50,60,187,1,1.148453,1,1,0,0,...,5.0,25.0,0,0.0,4,356.0,20150126.0,20150126.0,60900.0,61620.0
4,396,50,60,187,1,1.148453,1,1,0,0,...,6.0,29.0,0,3.0,4,313.0,20150829.0,20150829.0,54540.0,54840.0
5,636,67,46,417,1,1.794651,0,0,1,1,...,10.0,24.0,0,6.0,4,316.0,20141030.0,20141030.0,25200.0,25560.0
6,636,67,46,417,1,1.794651,0,0,1,1,...,5.0,NaN,0,4.0,4,345.0,20150807.0,20150807.0,61860.0,62220.0
7,636,67,46,417,1,1.794651,0,0,1,1,...,8.0,NaN,0,5.0,4,352.0,20141008.0,20141008.0,51000.0,51780.0
8,636,67,46,417,1,1.794651,0,0,1,1,...,5.0,21.0,0,1.0,4,339.0,20150821.0,20150821.0,28380.0,28920.0
9,636,67,46,417,1,1.794651,0,0,1,1,...,5.0,18.0,0,4.0,4,294.0,20150423.0,20150423.0,23760.0,24000.0


In [170]:
#q1 = q.dropna()
q1 = q.fillna(0)
#q1 = q.fillna(q.mean())
q1 = q1.dropna(axis=1, how='all')
#w = q.fillna(0)

In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

model = LinearRegression(n_jobs = -1)


In [26]:
rfe = RFE(model, 3)
x1 = q1.drop('duration', axis=1)
y1 = q1['duration']
x2 = q2.drop('duration', axis=1)
y2 = q2['duration']
#x3 = q3.drop('duration', axis=1)
#y3 = q3['duration']

In [26]:
from sklearn import linear_model

clf1 = linear_model.SGDRegressor(loss=  'squared_epsilon_insensitive')
clf1.fit(x1, y1)
clf2 = linear_model.SGDRegressor(loss=  'squared_epsilon_insensitive')
clf2.fit(x2, y2)
#clf3 = linear_model.SGDRegressor(loss=  'squared_epsilon_insensitive')
#clf3.fit(x3, y3)

SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_epsilon_insensitive', n_iter=5, penalty='l2',
       power_t=0.25, random_state=None, shuffle=True, verbose=0,
       warm_start=False)

In [27]:
clf1.coef_

array([  2.17814228e+12,   1.86596948e+12,  -2.25124693e+13,
        -7.06104793e+10,   5.70309831e+10,  -6.02359392e+10,
         5.73761106e+10,  -6.64443126e+10,   6.22414396e+12,
         2.71086431e+12,  -1.74624743e+12,  -1.33701800e+12,
        -3.01636836e+12,  -5.72231634e+12,  -6.95744187e+12,
        -1.21204045e+13,  -1.22310056e+13,   5.42903015e+10,
         6.08984671e+10,   8.40169632e+10,   1.39831882e+11,
         6.14413432e+11,   1.28813355e+12,   2.19842343e+12,
         1.44656840e+12,  -3.77196131e+12,  -1.86881441e+10,
        -2.74615001e+12,   3.57825296e+11,  -8.23796651e+12,
         2.46635959e+13,   2.93256705e+13,   2.73955123e+13,
        -3.26895485e+12])

In [28]:
clf2.coef_

array([ -5.75935738e+10,  -9.67104587e+10,  -4.05550810e+10,
         9.72666396e+10,   3.58098307e+10,   8.41933229e+10,
         3.46197402e+10,   8.19014246e+10,   6.89606186e+10,
         2.62667213e+09,  -1.88081759e+10,  -6.95067089e+10,
         4.24507349e+09,   1.01854757e+11,   3.74466643e+10,
        -8.66113017e+10,  -9.33088378e+10,  -5.20704719e+09,
        -4.01204055e+10,  -5.34708819e+10,  -6.75479904e+10,
         9.45453385e+10,  -1.11117509e+11,   6.47120977e+10,
        -1.25499971e+11,  -9.82495127e+10,  -3.01081945e+10,
         2.43057856e+10,   4.08233844e+10,   3.38566788e+10,
         1.28682167e+15,   1.29178776e+15,  -5.99177592e+13,
        -2.36237305e+13])

In [29]:
w = w.drop('duration', axis = 1)
t1 = w
t2 = w

In [30]:
clf1.predict(w)

array([  1.08793301e+21,   1.08738115e+21,   1.08790670e+21, ...,
        -1.43929497e+16,  -1.40299401e+16,  -1.36036482e+16])

In [31]:
clf2.predict(w)

array([  5.19610692e+22,   5.19347206e+22,   5.19592198e+22, ...,
        -3.76435010e+13,  -3.78386160e+13,  -3.96791963e+13])

In [32]:
w1 = rfe.fit(x1, y1)
w2 = rfe.fit(x2, y2)
#w3 = rfe.fit(x3, y3)

In [33]:
print (w1.n_features_)
print (w1.support_)
print (w1.ranking_)
print (w1.estimator_)

3
[False False False  True  True  True False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False]
[25 20 26  1  1  1  5  2 14 13 15  9 10 28 24 19 21  6  3  4 16 11 17 22 12
 18  7 23  8 29 31 30 32 27]
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)


In [89]:
q.columns


Index(['duration', 'start_station_id', 'end_station_id', 'bike_id',
       'subscription_type', 'start_station_lat', 'start_station_long',
       'end_station_lat', 'end_station_long', 'max_temperature_f',
       'mean_temperature_f', 'min_temperature_f', 'max_dew_point_f',
       'mean_dew_point_f', 'min_dew_point_f', 'max_humidity', 'mean_humidity',
       'min_humidity', 'max_sea_level_pressure_inches',
       'mean_sea_level_pressure_inches', 'min_sea_level_pressure_inches',
       'max_visibility_miles', 'mean_visibility_miles', 'min_visibility_miles',
       'max_wind_Speed_mph', 'mean_wind_speed_mph', 'max_gust_speed_mph',
       'precipitation_inches', 'cloud_cover', 'events', 'wind_dir_degrees',
       'start_date', 'end_date', 'start_hour', 'end_hour'],
      dtype='object')

In [34]:
print (w2.n_features_)
print (w2.support_)
print (w2.ranking_)
print (w2.estimator_)

3
[False False False  True  True  True False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False]
[25 20 26  1  1  1  5  2 14 13 15  9 10 28 24 19 21  6  3  4 16 11 17 22 12
 18  7 23  8 29 31 30 32 27]
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)


In [108]:
t1['duration'] = w1.predict(w)

In [109]:
t2['duration'] = w2.predict(w)

ValueError: X has a different shape than during fitting.

In [113]:
t1

,start_station_id,end_station_id,bike_id,subscription_type,start_station_lat,start_station_long,end_station_lat,end_station_long,max_temperature_f,mean_temperature_f,...,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,start_date,end_date,start_hour,end_hour,duration
0,50,60,187,1,37.795392,-122.394203,37.804770,-122.403234,84.0,72.0,...,29.0,0,2.0,4,272.0,20150827.0,20150827.0,516.0,523.0,396.289992
1,50,60,187,1,37.795392,-122.394203,37.804770,-122.403234,91.0,76.0,...,20.0,0,1.0,4,309.0,20140728.0,20140728.0,1326.0,1337.0,396.289992
2,50,60,187,1,37.795392,-122.394203,37.804770,-122.403234,87.0,74.0,...,0.0,0,0.0,4,349.0,20140609.0,20140609.0,522.0,527.0,396.289992
3,50,60,187,1,37.795392,-122.394203,37.804770,-122.403234,89.0,76.0,...,25.0,0,0.0,4,356.0,20150126.0,20150126.0,1015.0,1027.0,396.289992
4,50,60,187,1,37.795392,-122.394203,37.804770,-122.403234,92.0,78.0,...,29.0,0,3.0,4,313.0,20150829.0,20150829.0,909.0,914.0,396.289992
5,67,46,417,1,37.776619,-122.417385,37.795425,-122.404767,72.0,67.0,...,24.0,0,6.0,4,316.0,20141030.0,20141030.0,420.0,426.0,756.511441
6,67,46,417,1,37.776619,-122.417385,37.795425,-122.404767,78.0,70.0,...,0.0,0,4.0,4,345.0,20150807.0,20150807.0,1031.0,1037.0,756.511441
7,67,46,417,1,37.776619,-122.417385,37.795425,-122.404767,78.0,71.0,...,0.0,0,5.0,4,352.0,20141008.0,20141008.0,850.0,863.0,756.511441
8,67,46,417,1,37.776619,-122.417385,37.795425,-122.404767,79.0,70.0,...,21.0,0,1.0,4,339.0,20150821.0,20150821.0,473.0,482.0,756.511441
9,67,46,417,1,37.776619,-122.417385,37.795425,-122.404767,82.0,71.0,...,18.0,0,4.0,4,294.0,20150423.0,20150423.0,396.0,400.0,756.511441


In [117]:

test[(test['end_hour'] == 523.0) & (test['start_hour'] == 516.0)]

,duration,start_station_id,end_station_id,bike_id,subscription_type,start_station_lat,start_station_long,end_station_lat,end_station_long,max_temperature_f,...,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,start_date,end_date,start_hour,end_hour
160200,456,77,61,551,1,37.789625,-122.400811,37.780526,-122.390288,75.0,...,10.0,22.0,0,5.0,4,269.0,20131210.0,20131210.0,516.0,523.0
163389,934,70,60,422,1,37.776617,-122.395260,37.804770,-122.403234,65.0,...,7.0,13.0,0,7.0,4,304.0,20150528.0,20150528.0,516.0,523.0
221373,720,74,48,444,1,37.794139,-122.394434,37.799953,-122.398525,75.0,...,6.0,21.0,0,0.0,4,12.0,20140317.0,20140317.0,516.0,523.0
243987,355,76,50,621,1,37.786305,-122.404966,37.795392,-122.394203,75.0,...,9.0,NaN,0,4.0,4,331.0,20131021.0,20131021.0,516.0,523.0
271810,613,70,67,439,1,37.776617,-122.395260,37.776619,-122.417385,73.0,...,9.0,29.0,0,4.0,4,290.0,20150714.0,20150714.0,516.0,523.0
377919,763,70,51,578,1,37.776617,-122.395260,37.791464,-122.391034,63.0,...,3.0,14.0,0.01,6.0,4,250.0,20140524.0,20140524.0,516.0,523.0
382049,333,41,73,520,1,37.795001,-122.399970,37.798522,-122.407245,66.0,...,4.0,18.0,0,1.0,4,294.0,20150526.0,20150526.0,516.0,523.0
384972,7775,45,45,562,1,37.794231,-122.402923,37.794231,-122.402923,80.0,...,11.0,21.0,0,2.0,4,340.0,20150624.0,20150624.0,516.0,523.0


In [ ]:
w2.score(p2, test['duration'])

In [ ]:
print (w3.n_features_)
print (w3.support_)
print (w3.ranking_)
print (w3.estimator_)

In [35]:
2**-5

0.03125

In [26]:
q5 = q1
q5['is_true'] = np.random.uniform(0,1,len(q1)) <= 0.7
q5 = q5[q5['is_true']==True]
del q5['is_true']
x1 = q5.drop('duration'
             , axis=1)
y1 = q5['duration']
t = w.drop('duration', axis = 1)

M:\Users\Krion\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [27]:
len(x1.columns)

18

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

import time

for i in range(4):
    q5 = q1
    q5['is_true'] = np.random.uniform(0,1,len(q1)) <= 0.2
    q5 = q5[q5['is_true']==True]
    del q5['is_true']
    x1 = q5.drop('duration'
             , axis=1)
    y1 = q5['duration']
    t = w.drop('duration', axis = 1)

    print("[INFO] tuning hyperparameters via grid search")
    params = [
       {'C': [1,0.1,0.01,10,0.001], 'gamma': [0.001,0.01, 0.1,1,10], 'kernel': ['poly']},
    ]
    model =  SVR()
    grid = GridSearchCV(model, params, n_jobs = -1 )
    start = time.time()
    print("[INFO] fitting stage")

    acc = grid.fit(x1, y1).best_score_

    print("[INFO] grid search took {:.2f} seconds".format(time.time() - start))
    # evaluate the best grid searched model on the testing data
    #acc = grid.score(t, w['duration'])
    print("[INFO] grid search accuracy: {:.2f}%".format(acc * 100))
    print("[INFO] grid search best parameters: {}".format(grid.best_params_))


M:\Users\Krion\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[INFO] tuning hyperparameters via grid search
[INFO] fitting stage


In [36]:
t1 = test.fillna(0)

In [37]:
del t1['duration']

In [41]:
grid.predict(t1)

array([ 517.89858443,  517.89858443,  517.89859947, ...,  517.89858443,
        517.89858443,  517.89858443])

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVR

import time
for i in range(10):
    q5 = q1
    q5['is_true'] = np.random.uniform(0,1,len(q1)) <= 0.75
    q5 = q5[q5['is_true']==True]
    del q5['is_true']
    x1 = q5.drop('duration'
             , axis=1)
    y1 = q5['duration']
    t = w.drop('duration', axis = 1)
    print("[INFO] tuning hyperparameters via grid search")
    params = [
      {'C': [1,0.1,0.001,0.0001,10,100,1000,10000], 'epsilon': [0,0.01,0.1,1,2,4]},
    ]
    model =  LinearSVR()
    grid = GridSearchCV(model, params, n_jobs = -1)
    start = time.time()
    print("[INFO] fitting stage")

    acc = grid.fit(x1, y1).best_score_ 
    print("[INFO] grid search took {:.2f} seconds".format(time.time() - start))
    # evaluate the best grid searched model on the testing data
    print("[INFO] grid search accuracy: {:.2f}%".format(acc * 100))
    print("[INFO] grid search best parameters: {}".format(grid.best_params_))
    

[INFO] tuning hyperparameters via grid search
[INFO] fitting stage


In [ ]:
from sklearn.svm import SVR

model = SVR(kernel = 'rbf',C= 0.01, gamma = 1)
model.fit(x1, y1).score()
#predicted = model.predict(test['duration'])

In [114]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=10, n_iter=10, random_state=42)
a = svd.fit_transform(q1) 
print(svd.explained_variance_ratio_) 
print(svd.explained_variance_ratio_.sum()) 


[  3.47487754e-02   9.65178880e-01   5.77029225e-05   9.19253270e-06
   3.97554266e-06   8.37580828e-07   1.67801547e-07   1.63349086e-07
   9.00409509e-08   8.38533355e-08]
0.999999868839


In [117]:
b = svd.fit_transform(q['duration'])


IndexError: tuple index out of range

In [85]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=10, n_iter=10, random_state=42)
svd.fit(q2) 
print(svd.explained_variance_ratio_) 
print(svd.explained_variance_ratio_.sum()) 

[  9.99995881e-01   4.11847716e-06   5.35035525e-10   2.26960425e-10
   1.11738447e-10   2.67263461e-11   3.82206562e-12   3.28655336e-12
   2.81747822e-12   1.47477937e-12]
0.999999999996


In [ ]:
#Predict Output
predicted= model.predict(test)

In [52]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, perplexity=40, verbose=2)


In [55]:
q1['is_true'] = np.random.uniform(0,1,len(q1)) <= 0.2
q1 = q1
q1 = q1[q1['is_true']==True]


In [56]:
tsne1 = TSNE(n_components=2, perplexity=40, verbose=2).fit(q1)
#tsne2 = TSNE(n_components=2, perplexity=40, verbose=2).fit_transform(q2)
#tsne3 = TSNE(n_components=2, perplexity=40, verbose=2).fit_transform(q3)

[t-SNE] Computing pairwise distances...
[t-SNE] Computing 121 nearest neighbors...


MemoryError: 

In [ ]:
%matplotlib notebook
fig = figure(figsize=(10, 10))
ax = axes(frameon=False)
setp(ax, xticks=(), yticks=())
subplots_adjust(left=0.0, bottom=0.0, right=1.0, top=0.9,
                wspace=0.0, hspace=0.0)
scatter(tsne1[:, 0], tsne1[:, 1],
        c=newsgroups.target, marker="x")

In [ ]:
%matplotlib notebook
fig = figure(figsize=(10, 10))
ax = axes(frameon=False)
setp(ax, xticks=(), yticks=())
subplots_adjust(left=0.0, bottom=0.0, right=1.0, top=0.9,
                wspace=0.0, hspace=0.0)
scatter(tsne2[:, 0], tsne2[:, 1],
        c=newsgroups.target, marker="x")

In [ ]:
%matplotlib notebook
fig = figure(figsize=(10, 10))
ax = axes(frameon=False)
setp(ax, xticks=(), yticks=())
subplots_adjust(left=0.0, bottom=0.0, right=1.0, top=0.9,
                wspace=0.0, hspace=0.0)
scatter(tsne3[:, 0], tsne3[:, 1],
        c=newsgroups.target, marker="x")

In [158]:
#def f_regression(X,Y):
  #  return sklearn.feature_selection.f_classif(X,Y)
#featureSelector = sklearn.feature_selection.SelectKBest(score_func=f_regression,k= 10)
#featureSelector = featureSelector.fit(q,q['duration'])
#w = featureSelector.transform(q)
#featureSelector.scores_

M:\Users\Krion\Anaconda3\lib\site-packages\sklearn\feature_selection\univariate_selection.py:114: RuntimeWarning: divide by zero encountered in true_divide
  f = msb / msw


array([         inf,   8.15339544,   7.75772099,   6.05850363,
        98.52009901,  10.63189177,   8.82561818,  10.58749372,
         8.85980925,   4.35013718,   4.51343176,   4.46638633,
         2.78849521,   3.20358054,   3.47330677,   1.7025236 ,
         2.2528808 ,   3.0623593 ,   3.73018581,   3.70524678,
         3.70197321,   0.8239847 ,   1.78001684,   3.03465461,
         1.39393355,   2.61086947,   1.32091206,   1.87201574,
         3.36205388,   2.83049472,   1.78117961,   4.55231972,
         4.55231973,   4.11260847,   4.12655317])

In [159]:
w


array([[  3.96000000e+02,   5.00000000e+01,   6.00000000e+01, ...,
          3.78047700e+01,  -1.22403234e+02,   2.01508270e+07],
       [  3.96000000e+02,   5.00000000e+01,   6.00000000e+01, ...,
          3.78047700e+01,  -1.22403234e+02,   2.01407280e+07],
       [  3.96000000e+02,   5.00000000e+01,   6.00000000e+01, ...,
          3.78047700e+01,  -1.22403234e+02,   2.01406090e+07],
       ..., 
       [  2.93000000e+02,   7.40000000e+01,   6.00000000e+01, ...,
          3.78047700e+01,  -1.22403234e+02,   0.00000000e+00],
       [  2.93000000e+02,   7.40000000e+01,   6.00000000e+01, ...,
          3.78047700e+01,  -1.22403234e+02,   0.00000000e+00],
       [  2.93000000e+02,   7.40000000e+01,   6.00000000e+01, ...,
          3.78047700e+01,  -1.22403234e+02,   0.00000000e+00]])

In [28]:
q1['is_true'] = np.random.uniform(0,1,len(q1)) <= 0.2
q1 = q1
q1 = q1[q1['is_true']==True]


In [29]:
x1 = q1.drop('duration', axis=1)
y1 = q1['duration']

In [30]:

features = x1.columns[0:34]
features


Index(['start_station_id', 'end_station_id', 'bike_id', 'subscription_type',
       'start_station_lat', 'start_station_long', 'end_station_lat',
       'end_station_long', 'max_temperature_f', 'mean_temperature_f',
       'min_temperature_f', 'max_dew_point_f', 'mean_dew_point_f',
       'min_dew_point_f', 'max_humidity', 'mean_humidity', 'min_humidity',
       'max_sea_level_pressure_inches', 'mean_sea_level_pressure_inches',
       'min_sea_level_pressure_inches', 'max_visibility_miles',
       'mean_visibility_miles', 'min_visibility_miles', 'max_wind_Speed_mph',
       'mean_wind_speed_mph', 'max_gust_speed_mph', 'precipitation_inches',
       'cloud_cover', 'events', 'wind_dir_degrees', 'start_date', 'end_date',
       'start_hour', 'end_hour'],
      dtype='object')

In [31]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_jobs=-1, criterion = "entropy" )

clf.fit(x1[features], y1)

JoblibMemoryError: JoblibMemoryError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
M:\Users\Krion\Anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel.__main__', loader=<_f...da3\\lib\\site-packages\\ipykernel\\__main__.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
M:\Users\Krion\Anaconda3\lib\runpy.

In [ ]:
#clf.feature_importances_